# Julia Workshop: Optimization and Solvers

## @ CEF 2017

**Authors**: Chase Coleman and Spencer Lyon

**Date**: 27 June 2017


## Goal

A few different packages used for optimization or non-linear equation solving.


## QuantEcon

The QuantEcon library has a few simple optimizers and solvers. See the economics [notebook](economics.ipynb) for more information.

In [ ]:
# Pkg.add("QuantEcon")

## Optim.jl

Package for optimizing written in pure Julia

[Documentation](http://julianlsolvers.github.io/Optim.jl/stable/)

In [ ]:
# Pkg.add("Optim")

In [ ]:
using Optim

In [ ]:
rosenbrock(x) = (1.0 - x[1])^2 + 100.0*(x[2] - x[1]^2)^2

### Optimizing without gradient

In [ ]:
optimize(rosenbrock, zeros(2), NelderMead())

In [ ]:
optimize(rosenbrock, zeros(2), BFGS())

### Optimizing with gradient

In [ ]:
function rosenbrock_grad!(x::Vector, grad::Vector)
    grad[1] = -2.0*(1.0 - x[1]) - 400.0*(x[2] - x[1]^2)*x[1]
    grad[2] = 200.0*(x[2] - x[1]^2)
end

In [ ]:
optimize(rosenbrock, rosenbrock_grad!, zeros(2), LBFGS(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

In [ ]:
optimize(rosenbrock, rosenbrock_grad!, zeros(2), GradientDescent(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

### Optimizing with Hessian

In [ ]:
function rosenbrock_hess!(x::Vector, hess::Matrix)
    hess[1, 1] = 2.0 - 400.0 * x[2] + 1200.0*x[1]^2
    hess[1, 2] = -400.0 * x[1]
    hess[2, 1] = -400.0 * x[1]
    hess[2, 2] = 200.0
end

In [ ]:
optimize(rosenbrock, rosenbrock_grad!, rosenbrock_hess!, zeros(2), Newton())

## NLopt

Julia wrapper to high quality C library.

This library has _lots_ of options for algorithms. See [list](http://ab-initio.mit.edu/wiki/index.php/NLopt_Algorithms) of algorithms

[Julia package](https://github.com/JuliaOpt/NLopt.jl) and [C Documentation](http://ab-initio.mit.edu/wiki/index.php/NLopt)

In [ ]:
# Pkg.add("NLopt")

In [ ]:
using NLopt

In [ ]:
function rosenbrock_nlopt(x::Vector, grad::Vector)
    if length(grad) > 0
        rosenbrock_grad!(x, grad)
    end

    return rosenbrock(x)
end

### A non-gradient and gradient based method

In [ ]:
opt_LBFGS = Opt(:LD_LBFGS, 2)

min_objective!(opt_LBFGS, rosenbrock_nlopt)
xtol_rel!(opt_LBFGS, 1e-10)
ftol_rel!(opt_LBFGS, 1e-9)

NLopt.optimize(opt_LBFGS, zeros(2))

In [ ]:
opt_PRAXIS = Opt(:LN_PRAXIS, 2)

min_objective!(opt_PRAXIS, rosenbrock_nlopt)

NLopt.optimize(opt_PRAXIS, zeros(2))

### A constrained optimization problem

\begin{align*}
  \min_{x_1, x_2} &\sqrt{x_2} \\
  &\text{subject to } \\
  &x_2 \geq 0 \\
  &x_2 \geq (2 x_1)^3  \\
  &x_2 \geq (-x_1 + 1)^3
\end{align*}

In [ ]:
function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end

    return sqrt(x[1] + x[2])
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end

    return (a*x[1] + b)^3 - x[2]
end

In [ ]:
opt = Opt(:LD_MMA, 2)

lower_bounds!(opt, [-Inf, 0.])
xtol_rel!(opt, 1e-6)

min_objective!(opt, myfunc)

inequality_constraint!(opt, (x,g) -> myconstraint(x, g, 2.0, 0.0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x, g, -1.0 ,1.0), 1e-8)

(minf, minx, ret) = NLopt.optimize(opt, [1.234, 5.678])

println("got $minf at $minx (returned $ret)")

## NLsolve

Julia package written to solve systems of non-linear equations

[Documentation](https://github.com/JuliaNLSolvers/NLsolve.jl)

In [ ]:
# Pkg.add("NLsolve")

In [ ]:
using NLsolve

In [ ]:
function f!(xy::Vector, fxy::Vector)
    # Pull out arguments
    x, y = xy

    # Fill fxy
    fxy[1] = x^2 - sin(y)
    fxy[2] = y^2 - cos(x)
end

function g!(xy::Vector, jacxy::Matrix)
    x, y = xy
    # Fill with derivatives of first function
    jacxy[1, 1] = 2*x
    jacxy[1, 2] = -cos(y)

    # Fill off-diagonal
    jacxy[2, 1] = sin(x)
    jacxy[2, 2] = 2*y
end

In [ ]:
res = nlsolve(f!, g!, [0.0, 0.0], ftol=1e-10)